In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## Cargar embeddings y datos

In [2]:
bert_embeddings = np.load("../../Datos/goodreads_bert_embeddings.npy")  # shape: (num_items, embedding_dim)
#bert_embeddings_large = np.load("goodreads_bert_large_embeddings.npy")
num_items, embedding_dim = bert_embeddings.shape
#num_items, embedding_dim_large = bert_embeddings.shape

print(bert_embeddings.shape)

(4287, 768)


Crear matriz de similiradad entre embeddings de libros.

In [3]:
# computa similaridad coseno entre los features de las imagenes
cosSimilarities = cosine_similarity(bert_embeddings)
index = np.arange(num_items)
# guardamos los resultados en un dataframe
cos_similarities_df = pd.DataFrame(cosSimilarities, columns=index, index=index)
cos_similarities_df #.head()

,0,1,2,3,4,5,6,7,8,9,...,4277,4278,4279,4280,4281,4282,4283,4284,4285,4286
0,1.000000,0.673440,0.719119,0.764368,0.625514,0.653403,0.667970,0.746485,0.365827,0.778021,...,0.397503,0.543145,0.540901,0.755875,0.744101,0.626962,0.708529,0.717081,0.700909,0.717331
1,0.673440,1.000000,0.734144,0.676823,0.584083,0.681751,0.678472,0.818591,0.305292,0.753320,...,0.328485,0.502907,0.427954,0.732703,0.696381,0.539709,0.798792,0.728484,0.643580,0.654849
2,0.719119,0.734144,1.000000,0.791913,0.696762,0.760271,0.755001,0.834920,0.422391,0.775918,...,0.422090,0.613360,0.550323,0.777666,0.744185,0.652123,0.725647,0.753039,0.713980,0.772262
3,0.764368,0.676823,0.791913,1.000000,0.713837,0.798685,0.744020,0.781990,0.382928,0.733460,...,0.380449,0.556346,0.570946,0.814491,0.701956,0.643730,0.674895,0.639143,0.675953,0.768697
4,0.625514,0.584083,0.696762,0.713837,1.000000,0.656778,0.684132,0.681233,0.411072,0.590116,...,0.447254,0.538642,0.494055,0.700218,0.563526,0.562335,0.598830,0.552857,0.704690,0.624733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,0.626962,0.539709,0.652123,0.643730,0.562335,0.643197,0.631860,0.644756,0.382428,0.613364,...,0.493581,0.713897,0.709345,0.655685,0.630458,1.000000,0.591962,0.627551,0.596663,0.651534
4283,0.708529,0.798792,0.725647,0.674895,0.598830,0.714164,0.650955,0.707901,0.354210,0.728052,...,0.399784,0.568883,0.538148,0.669548,0.740572,0.591962,1.000000,0.804356,0.704511,0.707331
4284,0.717081,0.728484,0.753039,0.639143,0.552857,0.664127,0.692349,0.720891,0.365877,0.752773,...,0.383236,0.584921,0.542766,0.668235,0.781792,0.627551,0.804356,1.000000,0.660810,0.738834
4285,0.700909,0.643580,0.713980,0.675953,0.704690,0.657830,0.681003,0.771596,0.418413,0.720591,...,0.483352,0.589077,0.498116,0.694656,0.709605,0.596663,0.704511,0.660810,1.000000,0.625742


In [4]:
df_books = pd.read_csv('../../Datos/books.csv', sep=',')
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

In [5]:
import json
with open("../../Datos/goodreads_past_interactions.json", "r") as f:
    user_interactions = json.load(f)

idx2userid = {i: id_ for i, id_ in enumerate(user_interactions.keys())}
userid2idx = {id_:i for i, id_ in enumerate(user_interactions.keys())}

In [6]:
with open("../../Datos/goodreads_test_interactions.json", "r") as f:
    user_interactions_test = json.load(f)

Crear matrices sparse.

In [15]:
from scipy.sparse import coo_matrix
# convertimos diccionarios en matrices sparse (COO)
rows = []
cols = []
data = []

for user_id, book_ids in user_interactions.items():
    for book_id in book_ids:
        u_id = userid2idx[user_id]
        b_id = bookid2idx[book_id]
        rows.append(int(u_id))
        cols.append(int(b_id))
        data.append(1)

max_user_id = max(rows)
max_item_id = max(cols)

# Crear matriz sparse
user_item_matrix_train = coo_matrix((data, (rows, cols)), shape=(max_user_id + 1, max_item_id + 1))

rows_test = []
cols_test = []
data_test = []

for user_id, book_ids in user_interactions_test.items():
    for book_id in book_ids:
        u_id = userid2idx[user_id]
        b_id = bookid2idx[book_id]
        rows_test.append(int(u_id))
        cols_test.append(int(b_id))
        data_test.append(1)

max_user_id = max([max_user_id] + rows_test)
max_item_id = max([max_item_id] + cols_test)

test_users = list(set(rows_test))

# Crear matriz sparse
user_item_matrix_test = coo_matrix((data_test, (rows_test, cols_test)), shape=(max_user_id + 1, max_item_id + 1))
print(user_item_matrix_test.shape)

(52821, 4287)


In [8]:
def recommend(given_index, sim_matrix, nb_closest_books=10):

    closest_imgs = sim_matrix[given_index].sort_values(ascending=False)[1:nb_closest_books+1].index
    closest_imgs_scores = sim_matrix[given_index].sort_values(ascending=False)[1:nb_closest_books+1]

    closest_imgs = closest_imgs.to_numpy()
    closest_imgs_scores = closest_imgs_scores.to_numpy()
    recs = [(img, score) for img, score in zip(closest_imgs, closest_imgs_scores)]

    return recs

## Generar recomendaciones

Para cada usuario, generar recomendaciones basadas en similitud de embeddings para su historial de libros.

In [9]:
recs = []
counter = 0
nb_closest_books = 10  # number of closest books to recommend

for user in test_users:
  user = idx2userid[user]
  past_interactions = user_interactions[user]
  user_recs = []
  rec_tuples = set()
  # obtener libros similares para cada libro
  # con el que el usuario ha interactuado anteriormente
  for book_id in past_interactions:
    book_idx = bookid2idx[book_id]
    results = set(recommend(book_idx, cos_similarities_df, nb_closest_books)) # para sacar duplicados
    rec_tuples = rec_tuples.union(results)
  rec_tuples = list(rec_tuples)
  rec_tuples.sort(key=lambda x: x[1], reverse=True) # ordenar por similaridad
  rec_tuples = rec_tuples[:nb_closest_books] # quedarse con las primeras n
  for rec in rec_tuples:
    book_id = df_books.book_id[rec[0]]
    user_recs.append(bookid2idx[book_id])
  recs.append(user_recs)
  counter += 1
  #if counter > 20:
  #  break

print(recs[:2])

[[228, 2328, 1678, 2231, 2713, 879, 3070, 3791, 520, 1875], [2718, 2676, 2418, 3658, 3307, 704, 3092, 1643, 293, 4265]]


In [20]:
def rec_func(users, N=10):
  recs = []
  for user in users:
    user = idx2userid[user]
    past_interactions = user_interactions[user]
    user_recs = []
    rec_tuples = set()
    # obtener libros similares para cada libro
    # con el que el usuario ha interactuado anteriormente
    for book_id in past_interactions:
      book_idx = bookid2idx[book_id]
      results = set(recommend(book_idx, cos_similarities_df, N)) # para sacar duplicados
      rec_tuples = rec_tuples.union(results)
    rec_tuples = list(rec_tuples)
    rec_tuples.sort(key=lambda x: x[1], reverse=True) # ordenar por similaridad
    rec_tuples = rec_tuples[:N] # quedarse con las primeras n
    for rec in rec_tuples:
      book_id = df_books.book_id[rec[0]]
      user_recs.append(bookid2idx[book_id])
    recs.append(user_recs)
  return recs
        

## Generar recomendaciones con PCA

In [22]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
reduced_embeddings = pca.fit_transform(bert_embeddings)
# computa similaridad coseno entre los features de las imagenes
pca_cosSimilarities = cosine_similarity(reduced_embeddings)
index = np.arange(num_items)
# guardamos los resultados en un dataframe
pca_cos_similarities_df = pd.DataFrame(pca_cosSimilarities, columns=index, index=index)
pca_cos_similarities_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4277,4278,4279,4280,4281,4282,4283,4284,4285,4286
0,1.000000,-0.100883,0.009345,0.313633,0.078513,-0.191746,0.035630,0.102127,0.008421,0.373544,...,0.008368,-0.062376,-0.023754,0.261512,0.231845,0.034186,-0.022778,0.075509,0.139521,0.090402
1,-0.100883,1.000000,-0.111328,-0.157111,-0.195587,-0.215075,-0.081286,0.295434,-0.240070,0.096746,...,-0.227761,-0.288921,-0.433763,-0.017354,-0.082605,-0.369283,0.224100,-0.036724,-0.208072,-0.337063
2,0.009345,-0.111328,1.000000,0.261279,0.191619,0.050672,0.204004,0.268267,0.052698,0.094530,...,0.008651,-0.024256,-0.119933,0.128591,0.040065,-0.057310,-0.213590,0.035983,0.023125,0.145377
3,0.313633,-0.157111,0.261279,1.000000,0.313828,0.378756,0.230771,0.216389,0.011628,0.093685,...,-0.045824,-0.081388,0.048466,0.459135,0.018359,0.037429,-0.239268,-0.287274,-0.002466,0.244698
4,0.078513,-0.195587,0.191619,0.313828,1.000000,0.074414,0.272429,0.111265,0.212563,-0.162564,...,0.216830,0.044166,-0.038581,0.234960,-0.192558,-0.008152,-0.173455,-0.277036,0.362390,-0.013938


In [23]:
recs_pca = []
counter = 0
nb_closest_books = 10  # number of closest books to recommend

for user in test_users:
  user = idx2userid[user]
  past_interactions = user_interactions[user]
  user_recs = []
  rec_tuples = set()
  # obtener libros similares para cada libro
  # con el que el usuario ha interactuado anteriormente
  for book_id in past_interactions:
    book_idx = bookid2idx[book_id]
    results = set(recommend(book_idx, pca_cos_similarities_df, nb_closest_books)) # para sacar duplicados
    rec_tuples = rec_tuples.union(results)
  rec_tuples = list(rec_tuples)
  rec_tuples.sort(key=lambda x: x[1], reverse=True) # ordenar por similaridad
  rec_tuples = rec_tuples[:nb_closest_books] # quedarse con las primeras n
  for rec in rec_tuples:
    book_id = df_books.book_id[rec[0]]
    user_recs.append(bookid2idx[book_id])
  recs_pca.append(user_recs)
  counter += 1
  #if counter > 20:
  #  break

print(recs_pca[:2])

[[879, 2743, 3544, 2737, 4279, 439, 228, 3324, 784, 627], [2718, 2418, 2676, 3658, 3307, 3092, 1643, 704, 293, 2465]]


In [24]:
def rec_func_pca(users, N=10):
  recs = []
  for user in users:
    user = idx2userid[user]
    past_interactions = user_interactions[user]
    user_recs = []
    rec_tuples = set()
    # obtener libros similares para cada libro
    # con el que el usuario ha interactuado anteriormente
    for book_id in past_interactions:
      book_idx = bookid2idx[book_id]
      results = set(recommend(book_idx, pca_cos_similarities_df, N)) # para sacar duplicados
      rec_tuples = rec_tuples.union(results)
    rec_tuples = list(rec_tuples)
    rec_tuples.sort(key=lambda x: x[1], reverse=True) # ordenar por similaridad
    rec_tuples = rec_tuples[:N] # quedarse con las primeras n
    for rec in rec_tuples:
      book_id = df_books.book_id[rec[0]]
      user_recs.append(bookid2idx[book_id])
    recs.append(user_recs)
  return recs
        

## Evaluar recomendaciones

In [27]:
def custom_ranking_metrics_at_k(rec_func, train_user_items, test_user_items, K=10, show_progress=False):

    # Ensure matrices are in CSR format
    train_user_items = train_user_items.tocsr()
    test_user_items = test_user_items.tocsr()

    num_users, num_items = test_user_items.shape
    relevant = 0
    total_precision_div = 0
    total_map = 0
    total_ndcg = 0
    total_auc = 0
    total_users = 0

    # Compute cumulative gain for NDCG normalization
    cg = 1.0 / np.log2(np.arange(2, K + 2))  # Discount factor
    cg_sum = np.cumsum(cg)  # Ideal DCG normalization

    # Get users with at least one item in the test set
    users_with_test_data = np.where(np.diff(test_user_items.indptr) > 0)[0]

    # Progress bar
    #progress = tqdm.tqdm(total=len(users_with_test_data), disable=not show_progress)

    batch_size = 1000
    start_idx = 0
    #recs = []

    while start_idx < len(users_with_test_data):
        batch_users = users_with_test_data[start_idx:start_idx + batch_size]
        recommended_items = rec_func(batch_users, N=K)
        #recs.append(recommended_items)
        start_idx += batch_size

        for user_idx, user_id in enumerate(batch_users):
            test_items = set(test_user_items.indices[test_user_items.indptr[user_id]:test_user_items.indptr[user_id + 1]])

            if not test_items:
                continue  # Skip users without test data

            num_relevant = len(test_items)
            total_precision_div += min(K, num_relevant)

            ap = 0
            hit_count = 0
            auc = 0
            idcg = cg_sum[min(K, num_relevant) - 1]  # Ideal Discounted Cumulative Gain (IDCG)
            num_negative = num_items - num_relevant

            for rank, item in enumerate(recommended_items[user_idx]):
                if item in test_items:
                    relevant += 1
                    hit_count += 1
                    ap += hit_count / (rank + 1)
                    total_ndcg += cg[rank] / idcg
                else:
                    auc += hit_count  # Accumulate hits for AUC calculation

            auc += ((hit_count + num_relevant) / 2.0) * (num_negative - (K - hit_count))
            total_map += ap / min(K, num_relevant)
            total_auc += auc / (num_relevant * num_negative)
            total_users += 1

        #progress.update(len(batch_users))

    #progress.close()

    # Compute final metrics
    precision = relevant / total_precision_div if total_precision_div > 0 else 0
    mean_ap = total_map / total_users if total_users > 0 else 0
    mean_ndcg = total_ndcg / total_users if total_users > 0 else 0
    mean_auc = total_auc / total_users if total_users > 0 else 0

    return {
        "precision": precision,
        "map": mean_ap,
        "ndcg": mean_ndcg,
        "auc": mean_auc
    }

In [ ]:
metrics = custom_ranking_metrics_at_k(rec_func, user_item_matrix_train, user_item_matrix_test, K=10)
print(metrics)

metrics_pca = custom_ranking_metrics_at_k(rec_func_pca, user_item_matrix_train, user_item_matrix_test, K=10)
print(metrics_pca)

{'precision': 0.01, 'map': 0.0036563492063492065, 'ndcg': 0.010112269314444278, 'auc': 0.5038419452887536}
{'precision': 0.007, 'map': 0.002958333333333333, 'ndcg': 0.008227964083661044, 'auc': 0.5023420621931257}


In [15]:
print(recs_ranking_metrics_at_k(recs, user_item_matrix_test, K=10))
print(recs_ranking_metrics_at_k(recs_pca, user_item_matrix_test, K=10))

{'precision': 0.007, 'map': 0.00307420634920635, 'ndcg': 0.00606241562353928, 'auc': 0.502338555061959}
{'precision': 0.002, 'map': 0.0009, 'ndcg': 0.0022400556151517554, 'auc': 0.4998342296001867}
